In [1]:
from langchain import LLMChain, PromptTemplate
import openai, langchain
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.simplefilter('ignore')

In [2]:
load_dotenv(find_dotenv('newenv.env'))

True

In [3]:
langchain.__version__

'0.2.13'

In [4]:
openai.__version__

'1.40.6'

In [5]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
print(completion.choices[0].message.content)

In the realm of code, where logic takes flight,  
Dwells a concept, wondrous and slight.  
Recursion, the dance of a function so bold,  
A tale of itself, repeatedly retold.  

Picture a mirror, reflecting what's there,  
Each glance at itself, a familiar affair.  
A function calls forth, its own name in the chase,  
To solve a small puzzle, it takes a new face.  

Like nesting dolls in a vibrant embrace,  
Each one holds a story, a smaller trace.  
“Until a base case,” the wise coders say,  
For without it, our echoes will lead us astray.  

Down the rabbit hole, deeper we dive,  
With each invocation, we strive to survive.  
Break down the problem, don’t fear the descent,  
For every small step leads closer, we’re meant.  

A factorial’s charm, it shows what we seek,  
Five times four times three, down to just one, unique.  
So: five factorial calls four, who calls three,  
Each layer unfolds, till one’s left to see.  

Oh, cherish the beauty of recursion’s embrace,  
Where functions

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
messages= [
    ("system", "You are an expert assistant about movies."),
    ("human", "Can you tell top 5 movies of all time by revenue?")
]

print(llm.invoke(messages).content)

As of October 2023, the top five highest-grossing movies of all time by worldwide box office revenue are:

1. **Avatar** (2009) - Directed by James Cameron, this sci-fi epic has grossed over $2.9 billion.
2. **Avengers: Endgame** (2019) - Directed by Anthony and Joe Russo, this Marvel superhero film has grossed approximately $2.798 billion.
3. **Titanic** (1997) - Also directed by James Cameron, this romantic drama has grossed around $2.2 billion.
4. **Star Wars: The Force Awakens** (2015) - Directed by J.J. Abrams, this installment of the Star Wars saga has grossed about $2.068 billion.
5. **Avengers: Infinity War** (2018) - Directed by Anthony and Joe Russo, this film has grossed around $2.048 billion.

Please note that box office figures are constantly changing due to re-releases and new films, so it's always good to check for the most current information.


In [7]:
# from langchain.llms import OpenAI
# llm = OpenAI(model = "gpt-3.5-turbo-instruct", temperature=0) #gpt-4o-mini, gpt-3.5-turbo, gpt-4o are chat models so use ChatOpenAI for those

In [8]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Set the query you want to ask
input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

# Run the LLMChain to get the AI-generated answer
response = chain.run(input_data)

print("Question:", input_data["query"])
print("Answer:", response)

Question: What is the main advantage of quantum computing over classical computing?
Answer: The main advantage of quantum computing over classical computing is its ability to solve complex problems faster.


To create a PromptTemplate object, two arguments are required:

- input_variables: A list of variable names in the template; in this case, it includes only the query.
- template: The template string containing formatted text and placeholders.

After creating the PromptTemplate object, it can be used to produce prompts with specific questions by providing input data. The input data is a dictionary where the key corresponds to the variable name in the template. The resulting prompt can then be passed to a language model to generate answers.- 

# FewShotPromptTemplate

In [9]:
from langchain import FewShotPromptTemplate

In [10]:
examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

chain = LLMChain(llm=llm, prompt=dynamic_prompt)

input_data = {"input": "tiger"}
response = chain.run(input_data)

print(response)

Animal: tiger  
Habitat: tropical rainforests, grasslands, and mangrove swamps


Additionally, you can also save your PromptTemplate to a file in your local filesystem in JSON or YAML format:

     prompt_template.save("awesome_prompt.json")

and load it back 

    from langchain.prompts import load_prompt
    loaded_prompt = load_prompt("awesome_prompt.json")

In [12]:
#sarcasm example

examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "How can I learn quantum computing?"}
response = chain.run(input_data)

print(response)

Start by mastering the art of confusing your friends at parties. Then, dive into some textbooks that make you question your sanity!


Instead of utilizing the examples list of dictionaries directly, we can implement a **LengthBasedExampleSelector** like this:

In [22]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

In [29]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100
)

In [30]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

So, the dynamic_prompt_template utilizes the example_selector instead of a fixed list of examples. This allows the FewShotPromptTemplate to adjust the number of included examples based on the length of the input query. By doing so, it optimizes the use of the available context window and ensures that the language model receives an appropriate amount of contextual information.

In [31]:
chain = LLMChain(llm=llm, prompt=dynamic_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "Who invented the telephone?"}
response = chain.run(input_data)

print(response)

Alexander Graham Bell, the guy who really knew how to make a call!
